In [1]:

import codecs
import random
import re
import textwrap
from collections import defaultdict, deque, Counter
import math



* http://pit-claudel.fr/clement/blog/an-experimental-estimation-of-the-entropy-of-english-in-50-lines-of-python-code/#more-691
   
* Produces  entropy for model order = 1 for romeo.txt of 3.463 bits


In [2]:

def valid_probabilities(distribution):
    """Arguments:
            distribution:  a list of probabilities.
       returns:
            probabilities, list of valid probabilities. It removes zero values.

    All values should be positive.  The sum of the
    probabilities should be equal to 1.0 to 2 decimal places. """

    # Check the probabilities sum to 1.00 to 2 decimal places
    if round(sum(distribution), 2) != 1.0:
        raise ValueError('Probabilities do not sum to 1')

    # Check we have no negative values
    if min(distribution) < 0:
        raise ValueError('Negative probability')

    # Make a list of the probabilities in distribution where
    # the probability is greater than zero
    probabilities = [p for p in distribution if p > 0]

    return probabilities


In [3]:

def log2(posval):
    """Arguments:
            posval:  a postive number
       returns:
            result:  float, base 2 log of posval"""

    result = math.log(posval, 2)

    return result


In [4]:

def entropy_from_probabilities(probabilities):
    """Arguments:
            probabilities:  iterable, a probability distribution
       returns:
            HX:  float, the entropy of the passed distribution"""

    # Check we have a valid list of probabilities
    probabilities = valid_probabilities(probabilities)

    # Set entropy count to zero
    HX = 0

    # Loop over all probabilities and sum the entropy from each (Eq 2.52 P37)
    for px in probabilities:
        HX = HX + px * log2(1.0/px)

    return HX


In [5]:

def entropy_from_frequencies(frequencies):
    """Arguments:
            frequencies:  iterable, a frequency distribution
       returns:
            HX:  float, the entropy of the passed distribution"""

    # Transform the frequencies to probabilities. The probability of
    # an event is its frequency divided by the sum of all frequencies.

    sum_of_frequencies = sum(frequencies)

    # Make a list of : the probabilty for each frequency in the
    # list of frequencies.

    probabilities = [freq / float(sum_of_frequencies) for freq in frequencies]

    #
    # Now we have a probability distribution, we can find the entropy
    #
    HX = entropy_from_probabilities(probabilities)

    return HX



In [6]:

def entropy_rate(model):
    """Calculates the average entropy of the model data.  Does this by
    calculating the entropy for each prefix and weighting it by the
    frequency with which the prefix appears."""

    # Initialise counts
    total_freq       = 0
    weighted_entropy = 0

    # Loop for all prefixes
    for prefix in model:
        # The square brackets denote a list comprehension, which is read from
        # left to right.
        # In words : "Make a list of the frequencies for
        # each item beginning with prefix.
        freqs = [freq for freq in model[prefix].values()]

        # Calculate the total frequency of all tokens beginning with
        # this prefix
        prefix_freq = sum(freqs)

        # Increment the total frequency for the whole model
        total_freq += prefix_freq

        # Calculate the weighted entropy from this prefix.
        weighted_entropy += prefix_freq * entropy_from_frequencies(freqs)

    # Calculate the average entropy across all the prefix distributions
    return weighted_entropy / total_freq


In [7]:


def tokenize(file_path, tokenizer):
    """A generator function reads in the file at file_path and uses the passed
    tokenizer function to yield tokens.  Generator functions are 'lazy'.  They
    only do what work they have to do to yield the next value.  This means
    large files are handled with little memory use.

       Arguments:
            file_path:  string.  The path to the file to read in.
            tokenizer: function.  A function to split up the data into
                tokens
       returns:
            token:  the next token in the input"""

    # Open the file for reading.
    # break down each line in the file into tokens and yield them one at a
    # time.
    with codecs.open(file_path, mode="r", encoding="utf-8") as infile:
        for line in infile:
            for token in tokenizer(line.lower().strip()):
                yield token


In [8]:

def append_space(text):
    """Appends a space to a string"""

    return text + " "


In [9]:

def chars(file_path):
    """A function to read in data from a file and convert it to
    single characters
       Arguments:
            file_path:  string.  The path to the file to read in.
       returns:
            token:  the next character in the input"""

    #
    # tokenize will open and read file_path.  For each line it will
    # append a space and return its contents one character at a time.
    #
    return tokenize(file_path, append_space)


In [10]:

def break_into_words(text):
    """Function will break the text into words"""

    return re.findall(r"[a-zA-Z']+", text)


In [11]:

def words(file_path):
    """A function to read in data from a file and convert it to
    words
       Arguments:
            file_path:  string.  The path to the file to read in.
       returns:
            token:  the next word in the input"""

    #
    # tokenize will open and read file_path.  For each line it will
    # break the line into words and return them one at a time.
    #
    return tokenize(file_path, break_into_words)


In [12]:


def generate(model, length):
    """A function to create a random sample of length tokens (a token could be
    a word or character).
    After each iteration we modify the seed/state
    by removing the first element of the token and appending a random prefix
    from our model based on the new value of state."""

    text = []

    # Pick a random seed as a start point
    # e.g. For words with a prefix length of 3, this could be
    # "of each organic"
    prefix = seed(model)

    # Loop building up text
    for _ in range(length):

        # Store the first word/char of the current state
        # In our example, would be 'of' for first pass in our example
        text.append(prefix[0])

        # Pick a new word to append to the current prefix.  The pick function
        # looks at the distribution of words/chars starting with prefix
        # and picks one at random from the distribution
        # For example, or new prefix now drops the 'of' and may become
        # "each organic compound".
        # Then loop with this new prefix.
        prefix = prefix[1:] + (pick(model[prefix]), )

    return text


In [13]:


def pick(counter):
    """Randomly pick from our counter, weighted by frequency"""

    # Calc size of counter - the total of the frequencies
    
    size = sum( counter.values() )
    if size <= 0:
        print( counter )
        raise ValueError("No frequency values in passed counter")

    # Pick a random element as a target
    target = random.randint(1, size)

    # Step through the model unitil we reach/pass our target
    
    cumulative_frequency = 0
    
    for suffix, frequency in counter.items():
        cumulative_frequency += frequency

        # If we have reached our target, we are done
        if cumulative_frequency >= target:
            return suffix

    # If we get here we have not picked a suffix
    raise ValueError("Unable to obtain sample")



In [14]:


def seed(model):
    """Randomly pick a prefix from our model, weighted by frequency"""

    # Calc size of model
    size = sum([sum(p.values()) for p in model.values()])
    if size <= 0:
        raise ValueError("No frequency values in passed model")

    # Pick a random element
    target = random.randint(1, size)

    # Step through the model unitil we reach/pass our target
    cumulative_frequency = 0
    for prefix, possibles in model.items():
        cumulative_frequency += sum(possibles.values())

        # If we have reached our target, we are done
        if cumulative_frequency >= target:
            return prefix

    # If we get here, we have not found a seed.
    raise ValueError("Unable to obtain seed")


In [15]:


def markov_model(stream, model_order):
    """Function counts the frequency of all distinct strings in the stream
    beginning with a prefix of length model_order.  It returns a list of
    counters."""
    # Initialise our Counters.
    # model is a dictionary mapping (n?1)-character prefixes to a Counter;
    # that Counter maps each possible nth character to the number of times
    # this character followed the (n?1)-character prefix.
    # For example, model could be
    # model = {
    #          ('w', 'h'): {'y':25, 'o':12, 'a':16, ...},
    #          ('t', 'h'): {'i':15, 'a':18, 'e':34, ...},
    #          ...
    #         }
    # Making model a defaultdict (as opposed a simple dict, like model = {} )
    # means that we can just increment its count without first checking
    # if a key value exists.

    model = defaultdict( Counter )
    # Create a queue for appending each token we read.  We are using the deque
    # as a pipe of length model_order.
    # We add to the pipe by appending to it.
    #
    # Pipe State     Append
    # <empty>        D
    # D              O
    # DO             G
    # DOG            G
    # OGG            E
    # GGE            D
    # GED            etc.

    pipe = deque( maxlen=model_order )
    
    for token in stream:
        #
        # If the pipe holds model_order characters, then store it contents.
        #
        if len(pipe) == model_order:

            # Convert the pipe contents to something hashable, which can then
            # used as a dict key.  Do this with the tuple() function
            model[tuple(pipe)][token] += 1

        pipe.append(token)

    return model





## Main Loop


In [16]:

model_order = 3
sample_size = 300
    
filename = "romeo.txt"
   

In [17]:

## The entropy of characters

model = markov_model( chars(filename), model_order )


In [18]:

print('Model order = ', model_order)


Model order =  3


In [19]:

print("Letter Entropy:", entropy_rate( model ), ' bits/letter.')
    


Letter Entropy: 2.0192379863731866  bits/letter.


In [20]:

# Output the a random sample text generated from the input sample
# Format it as a block of chars width 70 (default for
# textwrap.fill()

print('Model letter output:')

res = textwrap.fill(  "".join( 
              generate( model, sample_size )
))


Model letter output:


In [21]:

print(  res  )


lone; thus of hat, go, juliet.  nurse ther.  rome sting!  laure, clock
and; an [with the and wellords: wherought or son, i'll to be gregory]
i do so life as needs! who i down!  lade mour shreath; thy fline othe
you; outh, confest, and me, thence thee.  nurse will mustly he's
lovert they a fived? the


In [22]:

# The entropy of words

model = markov_model(   words(filename), model_order   )


In [23]:


print( "Model order", model_order )
    
print(  "Word Entropy = :", entropy_rate(model), ' bits/word.'  )
    


Model order 3
Word Entropy = : 0.07689938101662905  bits/word.


In [24]:

# Output the a random sample text generated from the input sample
# Format it as a block of chars width 70 (default for
# textwrap.fill()

print( 'Model word output:' )

other_res = textwrap.fill(   
                " ".join(   generate(model, 100)    )  
)

print( other_res )


Model word output:
beauty thou art not well sweet sweet sweet nurse tell me what says my
love nurse your love says like an honest gentleman where is your
mother juliet where is my mother why she is within where should she be
how oddly thou repliest 'your love says like an honest gentleman where
is your mother juliet where is my page go villain fetch a surgeon exit
page romeo courage man the hurt cannot be much mercutio no 'tis not so
deep as a well nor so wide as a church door but 'tis enough 'twill
serve ask for me to
